# СЛОЖНЫЙ ФОРМАТ И ИХ ВСЕГО 8

In [22]:
import os
import csv
import wfdb
import heartpy as hp
import pandas as pd
import numpy as np
from scipy.signal import resample
import pickle

In [29]:
age_groups = {15:1,16:1,17:1,18:1,19:1,
             20:2,21:2,22:2,23:2,24:2,
             25:3,26:3,27:3,28:3,29:3,
             30:4,31:4,32:4,33:4,34:4,
             35:5,36:5,37:5,38:5,39:5,
             40:6,41:6,42:6,43:6,44:6,
             45:7,46:7,47:7,48:7,49:7,
             50:8,51:8,52:8,53:8,54:8,
             55:9,56:9,57:9,58:9,59:9,
             60:10,61:10,62:10,63:10,64:10,
             65:11,66:11,67:11,68:11,69:11,
             70:12,71:12,72:12,73:12,74:12,
             75:13,76:13,77:13,78:13,79:13,
             80:14,81:14,82:14,83:14,84:14,
             85:15,86:15,87:15,88:15,89:15,
             90:16,91:16,92:16,93:16,94:16,
             95:17,96:17,97:17,98:17,99:17,100:17,
             }

In [ ]:
with open("D:\\Proga\\AML\\datasets\\ppg-dalia\\data\\PPG_FieldStudy\\S1\\S1.pkl","rb") as f:
    data = pickle.load(f,encoding='latin1')

print(type(data))
print(data.keys())
# print(data['signal']["wrist"]["BVP"])
# print(len(data['signal']["wrist"]["BVP"]))
# print(data['label'])
# print(data['activity'])
# print(data['activity'])

# Получаем метки активности
activity_labels = data['activity']

# Получаем PPG-сигнал (BVP)
bvp_signal = data['signal']['wrist']['BVP']

# Получаем частоту сэмплирования PPG-сигнала
bvp_sampling_rate = 64  # Гц

# Определяем периоды времени, когда участник сидел (activity == 1)
rest_periods = np.where(activity_labels == 1)[0]

# Преобразуем индексы периодов покоя в диапазоны времени
rest_start_times = rest_periods[0::2]
rest_end_times = rest_periods[1::2]

# Извлекаем PPG-сигналы для периодов покоя
rest_bvp_signals = []
for start, end in zip(rest_start_times, rest_end_times):
    # Преобразуем индексы в временные метки
    start_time = start / 4  # Частота сэмплирования activity_labels - 4 Гц
    end_time = end / 4  # Частота сэмплирования activity_labels - 4 Гц
    
    # Преобразуем временные метки в индексы для PPG-сигнала
    start_index = int(start_time * bvp_sampling_rate)
    end_index = int(end_time * bvp_sampling_rate)
    
    # Извлекаем PPG-сигнал для периода покоя
    rest_bvp_signals.append(bvp_signal[start_index:end_index])

# Объединяем все PPG-сигналы в один массив
rest_bvp_signal = np.concatenate(rest_bvp_signals)

# Целевая частота дискретизации
target_fs = 1000

# Пересэмплирование PPG-сигнала до целевой частоты
num_samples = int(len(rest_bvp_signal) * target_fs / bvp_sampling_rate)
rest_bvp_signal_resampled = resample(rest_bvp_signal, num_samples).astype(np.float32)

# Обрезка сигнала до первых 5 минут (300 секунд)
five_minute_samples = 300 * target_fs
rest_bvp_signal_resampled = rest_bvp_signal_resampled[:five_minute_samples]

# Проверяем форму сигнала
print("Shape of rest_bvp_signal_resampled:", rest_bvp_signal_resampled.shape)

# Обработка сигнала с использованием heartpy
try:
    wd, m = hp.process(rest_bvp_signal_resampled.flatten(), target_fs)
    
    # Получаем анкетные данные
    questionnaire_data = data['questionnaire']
    age = questionnaire_data.get('age', None)
    print(age)
    # Определяем возрастную группу
    age_group = age_groups[age]
    
    # Создаем метаданные
    result_metadata = {
        'id': 1,  # Идентификатор записи
        'age_group': age_group,
        **m  # Метрики из heartpy
    }
    
    # Преобразуем метаданные в список словарей
    result_metadata_list = [result_metadata]

except hp.exceptions.BadSignalWarning as e:
    result_metadata = {
        'id': 1,
        'age_group': None,
        'error': "BadSignalWarning"
    }
    print("Error processing signal:", result_metadata)

except Exception as e:
    result_metadata = {
        'id': 1,
        'age_group': None,
        'error': str(e)
    }
    print("Error processing signal:", result_metadata)

<class 'dict'>
dict_keys(['rpeaks', 'signal', 'label', 'activity', 'questionnaire', 'subject'])
Shape of rest_bvp_signal_resampled: (300000, 1)
None
Error processing signal: {'id': 1, 'age_group': None, 'error': 'None'}


In [28]:
df_metadata = pd.DataFrame(result_metadata_list)
#df_metadata.dropna(inplace=True)
df_metadata

,id,age_group,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate
0,1,None,118.893659,504.652651,177.246809,126.770589,208.349559,0.916342,0.735409,147.0,147.311601,200.610141,92840.977347,0.734318,0.2


In [ ]:
df_signals = pd.DataFrame(result_signals)
df_signals

ValueError: Per-column arrays must each be 1-dimensional